# Observations:
    1. Charter school types had significantly better overall passing rates compared to district type schools.
    2. Higher spending per student did not generate higher passing rates.
    3. Smaller schools outperformed large schools in both reading and math passing percents, with math having the largest Delta.

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# Inputs
source1 = 'Resources/schools_complete.csv'
source2 = 'Resources/students_complete.csv'
school_df = pd.read_csv(source1, delimiter = ",")
student_df = pd.read_csv(source2, delimiter = ",")

# Join Data Sources
merged = pd.merge(student_df,school_df, on="school_name", how='left')

#Clean or Manipulate Data if required
def combined (rows):
    return (rows['math_score'] + rows['reading_score'])/2

merged['combined_avg_score'] = merged.apply(combined, axis = 1)
merged['math_pass_cnt'] = [1 if x >= 70 else 0 for x in merged['math_score']]
merged['reading_pass_cnt'] = [1 if y >= 70 else 0 for y in merged['reading_score']]
merged['combined_pass_cnt'] = [1 if z >= 70 else 0 for z in merged['combined_avg_score']]
mergedcopy = merged.copy()

# Create Summary for District Type Schools
district_df = mergedcopy.loc[mergedcopy['type'] == 'District',:]
DistrictGroup = district_df.groupby(['type'])
 
schools = DistrictGroup['School ID'].nunique()
students = DistrictGroup['Student ID'].count()
total_budget = (DistrictGroup['budget'].sum()/students).astype(int)
avg_math_score = (DistrictGroup['math_score'].mean()).round(2)
avg_reading_score = (DistrictGroup['reading_score'].mean()).round(2)
math_pass_percent = ((DistrictGroup['math_pass_cnt'].sum())/students*100).round(2)
reading_pass_percent = ((DistrictGroup['reading_pass_cnt'].sum())/students*100).round(2)
overall_pass_rate = ((DistrictGroup['combined_pass_cnt'].sum())/students*100).round(2)

district_overview_df = pd.DataFrame({
    "Total Schools" : schools,
    "Total Students" : students,
    "Total Budget" : total_budget,
    "Average Math Score" : avg_math_score,
    "Average Reading Score" : avg_reading_score,
    "% Passing Math" : math_pass_percent,
    "% Passing Reading" : reading_pass_percent,
    "Overall Passing Rate" : overall_pass_rate
})

print("District Summary:")
district_overview_df


District Summary:


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,,,,
District,7,26976,2611174,76.99,80.96,66.52,80.91,84.91


In [2]:
# School Summary
df2 = mergedcopy.rename(columns = {"school_name" : "School_Name","type": "School_Type"})
SchoolGroup = df2.groupby(['School_Name','School_Type'])
 
students = SchoolGroup['Student ID'].count()
school_budget = (SchoolGroup['budget'].sum()/students).astype(int)
student_budget = ((SchoolGroup['budget'].sum()/students)/students).astype(int)
avg_math_score = (SchoolGroup['math_score'].mean()).round(2)
avg_reading_score = (SchoolGroup['reading_score'].mean()).round(2)
math_pass_percent = ((SchoolGroup['math_pass_cnt'].sum())/students*100).round(2)
reading_pass_percent = ((SchoolGroup['reading_pass_cnt'].sum())/students*100).round(2)
overall_pass_rate = ((SchoolGroup['combined_pass_cnt'].sum())/students*100).round(2)

school_overview_df = pd.DataFrame({
    "Total Students" : students,
    "Total School Budget" : school_budget,
    "Per Student Budget" : student_budget,
    "Average Math Score" : avg_math_score,
    "Average Reading Score" : avg_reading_score,
    "% Passing Math" : math_pass_percent,
    "% Passing Reading" : reading_pass_percent,
    "Overall Passing Rate" : overall_pass_rate
})

print("School Summary:")
school_overview_df

School Summary:


,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School_Name,School_Type,,,,,,,,
Bailey High School,District,4976,3124928,628,77.05,81.03,66.68,81.93,85.19
Cabrera High School,Charter,1858,1081356,582,83.06,83.98,94.13,97.04,99.57
Figueroa High School,District,2949,1884411,639,76.71,81.16,65.99,80.74,84.67
Ford High School,District,2739,1763916,644,77.10,80.75,68.31,79.30,84.78
Griffin High School,Charter,1468,917500,625,83.35,83.82,93.39,97.14,99.46
Hernandez High School,District,4635,3022020,652,77.29,80.93,66.75,80.86,84.88
Holden High School,Charter,427,248087,581,83.80,83.81,92.51,96.25,98.59
Huang High School,District,2917,1910635,655,76.63,81.18,65.68,81.32,84.98
Johnson High School,District,4761,3094650,650,77.07,80.97,66.06,81.22,84.98


In [3]:
# Top 5 Performing Schools (By Passing Rate)

top5 = school_overview_df.sort_values("Overall Passing Rate" , ascending = False)
top52 = pd.DataFrame(top5.iloc[0:5,:])

print("Top 5 Performing Schools by Passing Rate")
top52


Top 5 Performing Schools by Passing Rate


,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School_Name,School_Type,,,,,,,,
Cabrera High School,Charter,1858,1081356,582,83.06,83.98,94.13,97.04,99.57
Griffin High School,Charter,1468,917500,625,83.35,83.82,93.39,97.14,99.46
Shelton High School,Charter,1761,1056600,600,83.36,83.73,93.87,95.85,99.38
Wilson High School,Charter,2283,1319574,578,83.27,83.99,93.87,96.54,99.26
Wright High School,Charter,1800,1049400,583,83.68,83.96,93.33,96.61,99.22


In [4]:
# Bottom 5 Performing Schools (By Passing Rate)

bottom5 = school_overview_df.sort_values("Overall Passing Rate" , ascending = True)
bottom52 = pd.DataFrame(bottom5.iloc[0:5,:])

print("Bottom 5 Performing Schools by Passing Rate")
bottom52

Bottom 5 Performing Schools by Passing Rate


,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School_Name,School_Type,,,,,,,,
Figueroa High School,District,2949,1884411,639,76.71,81.16,65.99,80.74,84.67
Rodriguez High School,District,3999,2547363,637,76.84,80.74,66.37,80.22,84.75
Ford High School,District,2739,1763916,644,77.10,80.75,68.31,79.30,84.78
Hernandez High School,District,4635,3022020,652,77.29,80.93,66.75,80.86,84.88
Huang High School,District,2917,1910635,655,76.63,81.18,65.68,81.32,84.98


In [5]:
### Math Scores by Grade and school
df2['grade'] = pd.Categorical(df2['grade'], categories = ['9th','10th','11th','12th'])
ByGrade = df2.sort_values('grade' , ascending = True)
Scores = ByGrade.groupby(['School_Name','grade'])
avg_math_score = (Scores['math_score'].mean()).round(2)
MathScore_df = pd.DataFrame({"Average Math Score" : avg_math_score})
MathScore_df

Average Math Score
School_Name           grade                    
Bailey High School    9th                 77.08
                      10th                77.00
                      11th                77.52
                      12th                76.49
Cabrera High School   9th                 83.09
                      10th                83.15
                      11th                82.77
                      12th                83.28
Figueroa High School  9th                 76.40
                      10th                76.54
                      11th                76.88
                      12th                77.15
Ford High School      9th                 77.36
                      10th                77.67
                      11th                76.92
                      12th                76.18
Griffin High School   9th                 82.04
                      10th                84.23
                      11th                83.84
                      12th                83.36
Hernandez High School 9th                 77.44
                      10th                77.34
                      11th                77.14
                      12th                77.19
Holden High School    9th                 83.79
                      10th                83.43
                      11th                85.00
                      12th                82.86
Huang High School     9th                 77.03
                      10th                75.91
                      11th                76.45
                      12th                77.23
Johnson High School   9th                 77.19
                      10th                76.69
                      11th                77.49
                      12th                76.86
Pena High School      9th                 83.63
                      10th                83.37
                      11th                84.33
                      12th                84.12
Rodriguez High School 9th                 76.86
                      10th                76.61
                      11th                76.40
                      12th                77.69
Shelton High School   9th                 83.42
                      10th                82.92
                      11th                83.38
                      12th                83.78
Thomas High School    9th                 83.59
                      10th                83.09
                      11th                83.50
                      12th                83.50
Wilson High School    9th                 83.09
                      10th                83.72
                      11th                83.20
                      12th                83.04
Wright High School    9th                 83.26
                      10th                84.01
                      11th                83.84
                      12th                83.64

In [6]:
### Reading Scores by Grade and School
avg_reading_score = (Scores['reading_score'].mean()).round(2)
ReadScore_df = pd.DataFrame({"Average Reading Score" : avg_reading_score})
ReadScore_df

Average Reading Score
School_Name           grade                       
Bailey High School    9th                    81.30
                      10th                   80.91
                      11th                   80.95
                      12th                   80.91
Cabrera High School   9th                    83.68
                      10th                   84.25
                      11th                   83.79
                      12th                   84.29
Figueroa High School  9th                    81.20
                      10th                   81.41
                      11th                   80.64
                      12th                   81.38
Ford High School      9th                    80.63
                      10th                   81.26
                      11th                   80.40
                      12th                   80.66
Griffin High School   9th                    83.37
                      10th                   83.71
                      11th                   84.29
                      12th                   84.01
Hernandez High School 9th                    80.87
                      10th                   80.66
                      11th                   81.40
                      12th                   80.86
Holden High School    9th                    83.68
                      10th                   83.32
                      11th                   83.82
                      12th                   84.70
Huang High School     9th                    81.29
                      10th                   81.51
                      11th                   81.42
                      12th                   80.31
Johnson High School   9th                    81.26
                      10th                   80.77
                      11th                   80.62
                      12th                   81.23
Pena High School      9th                    83.81
                      10th                   83.61
                      11th                   84.34
                      12th                   84.59
Rodriguez High School 9th                    80.99
                      10th                   80.63
                      11th                   80.86
                      12th                   80.38
Shelton High School   9th                    84.12
                      10th                   83.44
                      11th                   84.37
                      12th                   82.78
Thomas High School    9th                    83.73
                      10th                   84.25
                      11th                   83.59
                      12th                   83.83
Wilson High School    9th                    83.94
                      10th                   84.02
                      11th                   83.76
                      12th                   84.32
Wright High School    9th                    83.83
                      10th                   83.81
                      11th                   84.16
                      12th                   84.07

In [11]:
### Scores by School Spending
df4 = school_overview_df.sort_values(["Per Student Budget","Overall Passing Rate"], ascending = False)
spendingavg = df4.groupby(["Per Student Budget", 'School_Name'])

avg_math_score = spendingavg["Average Math Score"].mean()
avg_reading_score = spendingavg["Average Reading Score"].mean()
math_pass_percent = spendingavg["% Passing Math"].mean()
reading_pass_percent = spendingavg["% Passing Reading"].mean()
overall_pass_rate = spendingavg["Overall Passing Rate"].mean()

spending_overview_df = pd.DataFrame({
    "Average Math Score" : avg_math_score,
    "Average Reading Score" : avg_reading_score,
    "% Passing Math" : math_pass_percent,
    "% Passing Reading" : reading_pass_percent,
    "Overall Passing Rate" : overall_pass_rate
})
spending_overview_df

,,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,School_Name,,,,,
578,Wilson High School,83.27,83.99,93.87,96.54,99.26
581,Holden High School,83.80,83.81,92.51,96.25,98.59
582,Cabrera High School,83.06,83.98,94.13,97.04,99.57
583,Wright High School,83.68,83.96,93.33,96.61,99.22
600,Shelton High School,83.36,83.73,93.87,95.85,99.38
609,Pena High School,83.84,84.04,94.59,95.95,99.17
625,Griffin High School,83.35,83.82,93.39,97.14,99.46
628,Bailey High School,77.05,81.03,66.68,81.93,85.19
637,Rodriguez High School,76.84,80.74,66.37,80.22,84.75


In [24]:
### Scores by School Size
bins = [0,1799,2940,5000]
size = ["Small","Medium","Large"]
df2['School_Size']=pd.cut(df2['size'],bins,labels=size)

SizeGroup = df2.groupby(['School_Size'])
 
students = SizeGroup['Student ID'].count()
avg_math_score = (SizeGroup['math_score'].mean()).round(2)
avg_reading_score = (SizeGroup['reading_score'].mean()).round(2)
math_pass_percent = ((SizeGroup['math_pass_cnt'].sum())/students*100).round(2)
reading_pass_percent = ((SizeGroup['reading_pass_cnt'].sum())/students*100).round(2)
overall_pass_rate = ((SizeGroup['combined_pass_cnt'].sum())/students*100).round(2)

size_df = pd.DataFrame({
    "Average Math Score" : avg_math_score,
    "Average Reading Score" : avg_reading_score,
    "% Passing Math" : math_pass_percent,
    "% Passing Reading" : reading_pass_percent,
    "Overall Passing Rate" : overall_pass_rate
})

size_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School_Size,,,,,
Small,83.48,83.83,93.62,96.58,99.23
Medium,80.17,82.51,80.70,88.73,92.29
Large,77.02,80.96,66.40,81.06,84.92


In [26]:
### Scores by School Type
TypeGroup = df2.groupby(['School_Type'])
 
students = TypeGroup['Student ID'].count()
avg_math_score = (TypeGroup['math_score'].mean()).round(2)
avg_reading_score = (TypeGroup['reading_score'].mean()).round(2)
math_pass_percent = ((TypeGroup['math_pass_cnt'].sum())/students*100).round(2)
reading_pass_percent = ((TypeGroup['reading_pass_cnt'].sum())/students*100).round(2)
overall_pass_rate = ((TypeGroup['combined_pass_cnt'].sum())/students*100).round(2)

type_df = pd.DataFrame({
    "Average Math Score" : avg_math_score,
    "Average Reading Score" : avg_reading_score,
    "% Passing Math" : math_pass_percent,
    "% Passing Reading" : reading_pass_percent,
    "Overall Passing Rate" : overall_pass_rate
})

type_df



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School_Type,,,,,
Charter,83.41,83.90,93.70,96.65,99.29
District,76.99,80.96,66.52,80.91,84.91
